In [1]:
# !pip3 install torch torchvision torchaudio

In [2]:
from transformers import GPT2Tokenizer

# Initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add all single characters as tokens
tokenizer.add_tokens(list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.,!? "))


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

## Fine tuning

In [3]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Load pre-trained model
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# Adjust model for new character-level tokenizer
model.resize_token_embeddings(len(tokenizer))






Embedding(50258, 768)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=0, #1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

In [3]:
len(tokenizer)

50258

### Create a Character level dataset

In [12]:
import json

# Step 1: Alphabet string
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
eos_token = "<EOS>"

# Step 2: Generate input-output pairs
dataset = []
for i in range(1, len(alphabet)):
    input_seq = alphabet[:i]
    target_char = alphabet[i]
    dataset.append({"input": input_seq, "target": target_char})

# Add the EOS token after "Z"
dataset.append({"input": alphabet, "target": eos_token})

# Step 3: Print the dataset
for data in dataset:
    print(data)

# Optional: Save the dataset to a file (JSON format)
with open('alphabet_dataset.json', 'w') as f:
    json.dump(dataset, f)


{'input': 'A', 'target': 'B'}
{'input': 'AB', 'target': 'C'}
{'input': 'ABC', 'target': 'D'}
{'input': 'ABCD', 'target': 'E'}
{'input': 'ABCDE', 'target': 'F'}
{'input': 'ABCDEF', 'target': 'G'}
{'input': 'ABCDEFG', 'target': 'H'}
{'input': 'ABCDEFGH', 'target': 'I'}
{'input': 'ABCDEFGHI', 'target': 'J'}
{'input': 'ABCDEFGHIJ', 'target': 'K'}
{'input': 'ABCDEFGHIJK', 'target': 'L'}
{'input': 'ABCDEFGHIJKL', 'target': 'M'}
{'input': 'ABCDEFGHIJKLM', 'target': 'N'}
{'input': 'ABCDEFGHIJKLMN', 'target': 'O'}
{'input': 'ABCDEFGHIJKLMNO', 'target': 'P'}
{'input': 'ABCDEFGHIJKLMNOP', 'target': 'Q'}
{'input': 'ABCDEFGHIJKLMNOPQ', 'target': 'R'}
{'input': 'ABCDEFGHIJKLMNOPQR', 'target': 'S'}
{'input': 'ABCDEFGHIJKLMNOPQRS', 'target': 'T'}
{'input': 'ABCDEFGHIJKLMNOPQRST', 'target': 'U'}
{'input': 'ABCDEFGHIJKLMNOPQRSTU', 'target': 'V'}
{'input': 'ABCDEFGHIJKLMNOPQRSTUV', 'target': 'W'}
{'input': 'ABCDEFGHIJKLMNOPQRSTUVW', 'target': 'X'}
{'input': 'ABCDEFGHIJKLMNOPQRSTUVWX', 'target': 'Y'}
{'in

### Convert to pytorch dataset

In [22]:
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments

# # Step 1: Define your dataset
# dataset = [
#     {"input": "A", "target": "B"},
#     {"input": "AB", "target": "C"},
#     {"input": "ABC", "target": "D"},
#     # Add more sequences as needed
#     {"input": "WXYZ", "target": "<EOS>"}
# ]

# Step 2: Load the tokenizer and add special tokens
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add special tokens (pad_token and eos_token)
tokenizer.add_special_tokens({
    'pad_token': '[PAD]',
    'eos_token': '<EOS>'
})

# Step 3: Define a PyTorch Dataset class
class CharPredictionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        input_seq = sample['input']
        target_char = sample['target']

        # Tokenize input sequence and pad to max_length
        input_ids = self.tokenizer(input_seq, padding='max_length', max_length=self.max_length, return_tensors="pt").input_ids.squeeze()

        # Tokenize target character (the next character or EOS)
        target_ids = self.tokenizer(target_char, padding='max_length', max_length=1, return_tensors="pt").input_ids.squeeze()

        # Return a dictionary compatible with Hugging Face Trainer
        return {
            'input_ids': input_ids,
            'labels': target_ids
        }






In [25]:
# Step 4: Instantiate your dataset
train_dataset = CharPredictionDataset(data=dataset, tokenizer=tokenizer, max_length=26)
# Step 5: Define model and resize token embeddings (to account for the new tokens added)
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrator\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Embedding(50259, 768)

In [29]:

# Step 6: Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,  # Adjust based on your system's memory
    save_steps=10_000,
    save_total_limit=2,
)

In [31]:



# Step 7: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Step 8: Train the model
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: Expected input batch_size (100) to match target batch_size (3).

In [10]:
input_text = "Hello, world!"
characters = list(input_text)

model.eval()
input_ids = tokenizer("Hi, How are y", return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=50,
    do_sample=True,      # Enable sampling
    top_k=20,            # Consider top 50 tokens for sampling
    temperature=0.7      # Control randomness
    )
predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(predicted_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


H i ,   H o w   a r e   y                                                                          
